In [4]:
from torchtext.vocab import build_vocab_from_iterator
from datasets import load_dataset

M:\disco M\Python\venvs\env_torch\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# baixando a base utilizada
dataset = load_dataset("AiresPucrs/google-play-apps-review-pt")

In [6]:
from torchtext.data.utils import get_tokenizer

In [7]:
tokenizer = get_tokenizer('spacy', language='pt_core_news_sm')
tokenizer('Olá você!')

['Olá', 'você', '!']

In [34]:
from typing import Callable, List, Iterable
# def yield_tokens(data_iter: Iterable, tokenizer: Callable[str, List[str]]) -> List[str]:
def yield_tokens(data_iter: Iterable, tokenizer) -> List[str]:
    for data_sample in data_iter['train']['review']:
        yield tokenizer(data_sample)

In [33]:
for text in dataset['train']['review']:
    print(text)
    break

o aplicativo e bom disparadamente melhor que o concorrente whatsapp pontos positivos  possibilidade de aplicar temas personalizados para sair da aparencia padrao de acordo com o usuario  a nao utilizacao de um backup local e sem a possibilidade de perder todas as mensagens acidentalmente por ser um servico via nuvem  a possibilidade de usar bots como um diferencial alem de somente usar o aplicativo para conversar ou seja e possivel ampliar o uso do aplicativo para outras coisas interessantes como por exemplo estudar  a possibilidade de se entreter com jogos e se divertir com outros contatosamigos similar ao ponto anterior  a existencia de um chat secreto para autodestruir mensagens que 2 usuarios nao queiram que fiquem armazenadas na nuvem sendo assim uma forma de conversar com privacidade total ainda ha outros pontos positivos mas nao e necessario citar todos eu tenho somente um ponto negativo tal ponto e a instabilidade do sistema em nuvem do telegram que certas vezes dessincroniza a

In [35]:
UNK_IDX, PAD_IDX, BOS_IDX, EOS_IDX = 0, 1, 2, 3
special_symbols = ['<unk>', '<pad>', '<sos>', '<eos>']

In [36]:
vocab_transform = build_vocab_from_iterator(
    yield_tokens(dataset, tokenizer),
    min_freq=2,
    specials=special_symbols,
    special_first=True
)

In [37]:
vocab_transform.set_default_index(UNK_IDX)

In [88]:
print(vocab_transform.get_itos())

['<unk>', '<pad>', '<sos>', '<eos>', 'e', 'o', 'de', 'que', 'nao', 'a', 'app', 'um', 'do', 'com', 'para', 'eu', 'no', 'mais', 'muito', 'em', 'uma', 'mas', 'tem', 'por', 'os', 'aplicativo', 'se', 'da', 'pra', 'as', 'isso', 'na', 'so', 'ja', 'pedido', 'meu', 'quando', 'esta', 'bom', ' ', 'como', 'me', 'ou', 'minha', 'sem', 'mesmo', 'ele', 'foi', 'vezes', 'ter', 'problema', 'nada', 'fazer', 'opcao', 'ao', 'conta', 'ate', 'esse', 'pelo', 'ser', 'nem', 'videos', 'pois', 'estou', 'sempre', 'q', 'entrega', 'tenho', 'bem', 'fica', 'consigo', 'voce', 'agora', 'sao', 'video', 'tempo', 'dos', 'tudo', 'pessoas', 'porem', 'melhor', 'depois', 'vez', 'usar', 'ainda', 'porque', 'uber', 'tambem', 'tive', 'ver', 'cartao', 'vc', 'valor', 'assim', 'ta', 'nos', 'nunca', 'fiz', 'pagamento', 'hora', 'restaurante', 'eles', 'era', 'acho', 'problemas', 'aparece', 'coisa', 'essa', 'pq', 'otimo', 'vou', 'outro', 'compra', 'suporte', 'dia', 'todos', 'das', 'estao', 'vai', 'ai', 'dinheiro', 'celular', 'atualizacao'

In [39]:
# print(vocab_transform.get_itos())

In [13]:
from torch import Tensor
import torch
import torch.nn as nn
import math
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [50]:
class PositionalEncoding(nn.Module):
    def __init__(self,
                 d_model: int,
                 dropout: float,
                 maxlen: int = 5000):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, d_model, 2)* math.log(10000) / d_model)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, d_model))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding: Tensor):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [51]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size: int, d_model):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, d_model)
        self.d_model = d_model

    def forward(self, tokens: Tensor):
        return self.embedding(tokens.long()) * math.sqrt(self.d_model)

In [60]:
class DecoderBlock(nn.Module):
    def __init__(
        self,
        d_model: int = 256,
        nhead: int = 8,
        num_decoder_layers: int = 3,
        dim_feedforward: int = 512,
        dropout: float = 0.1
    ):
        super(DecoderBlock, self).__init__()
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=d_model,
            nhead=nhead,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        
        self.decoder = nn.TransformerDecoder(self.decoder_layer, num_layers=num_decoder_layers)
    def forward(
        self,
        tgt,
        memory,
        tgt_mask=None,
        memory_mask=None,
        tgt_key_padding_mask=None,
        memory_key_padding_mask=None
    ):
        output = self.decoder(
            tgt,
            memory, 
            tgt_mask=tgt_mask,
            memory_mask=memory_mask,
            tgt_key_padding_mask=tgt_key_padding_mask, 
            memory_key_padding_mask=memory_key_padding_mask
        )
        return output

In [53]:
torch.manual_seed(42)

In [67]:
124/4

31.0

In [75]:
# configs
NUM_DECODER_LAYERS = 3
DIM_FF = 512
D_MODEL = 256
N_HEADS = 8
VOCAB_SIZE = len(vocab_transform)
DROPOUT = 0.1

In [76]:
class GooglePlayModel(nn.Module):
    def __init__(
        self,
        num_decoder_layers: int = NUM_DECODER_LAYERS,
        d_model: int = D_MODEL,
        nhead: int = N_HEADS,
        vocab_size: int = VOCAB_SIZE,
        dim_feedforward: int = DIM_FF,
        dropout: float = DROPOUT
    ):
        super(GooglePlayModel, self).__init__()
        self.positional_encoding = PositionalEncoding(d_model=d_model, dropout=dropout)
        self.embedding = TokenEmbedding(vocab_size=vocab_size, d_model=d_model)
        self.decoder = DecoderBlock(
            d_model=d_model,
            nhead=nhead,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )
        self.generator = nn.Linear(d_model, vocab_size)

    def forward(self, x, mask, memory_mask):
        pos_enc = self.positional_encoding(self.embedding(x))
        decoder_output = self.decoder(pos_enc, tgt_mask=mask, memory_mask=memory_mask)
        output = self.generator(decoder_output)

In [77]:
google_decoder = GooglePlayModel().to(DEVICE)

In [78]:
from torchinfo import summary

In [79]:
summary(google_decoder, verbose=0)

Layer (type:depth-idx)                                                 Param #
GooglePlayModel                                                        --
├─PositionalEncoding: 1-1                                              --
│    └─Dropout: 2-1                                                    --
├─TokenEmbedding: 1-2                                                  --
│    └─Embedding: 2-2                                                  3,940,352
├─DecoderBlock: 1-3                                                    --
│    └─TransformerDecoderLayer: 2-3                                    --
│    │    └─MultiheadAttention: 3-1                                    263,168
│    │    └─MultiheadAttention: 3-2                                    263,168
│    │    └─Linear: 3-3                                                131,584
│    │    └─Dropout: 3-4                                               --
│    │    └─Linear: 3-5                                                131,328
│    │

In [80]:
allocated_memory = torch.cuda.memory_allocated()
print("Memória alocada na GPU:", allocated_memory / 1024**2, "MB")

Memória alocada na GPU: 117.09423828125 MB


In [85]:
# testando o decoder
decoder_layer = nn.TransformerDecoderLayer(d_model=512, nhead=8)
transformer_decoder = nn.TransformerDecoder(decoder_layer, num_layers=6)
# memory = torch.rand(10, 32, 512)
tgt = torch.rand(20, 32, 512)
# out = transformer_decoder(tgt, memory)
out = transformer_decoder(tgt)

TypeError: forward() missing 1 required positional argument: 'memory'

In [86]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import math

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout, max_len=5000):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)

        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0).transpose(0, 1)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

class Seq2SeqModel(nn.Module):
    def __init__(self, output_dim, d_model, nhead, num_decoder_layers, dim_feedforward, dropout):
        super(Seq2SeqModel, self).__init__()
        
        self.decoder_embedding = nn.Embedding(output_dim, d_model)
        self.positional_encoding = PositionalEncoding(d_model, dropout)
        
        decoder_layer = nn.TransformerDecoderLayer(d_model, nhead, dim_feedforward, dropout)
        self.decoder = nn.TransformerDecoder(decoder_layer, num_layers=num_decoder_layers)
        
        self.fc_out = nn.Linear(d_model, output_dim)

    def make_trg_mask(self, trg):
        trg_pad_mask = (trg.transpose(0, 1) == 0).unsqueeze(-2)
        trg_len = trg.shape[0]
        trg_sub_mask = torch.tril(torch.ones((trg_len, trg_len), device=trg.device)).bool()
        trg_mask = trg_pad_mask & trg_sub_mask
        return trg_mask
    
    def forward(self, memory, trg):
        trg_mask = self.make_trg_mask(trg)
        
        trg = self.decoder_embedding(trg) * math.sqrt(memory.size(-1))
        trg = self.positional_encoding(trg)
        
        output = self.decoder(tgt=trg, memory=memory, tgt_mask=trg_mask)
        output = self.fc_out(output)
        
        return output

# Define hyperparameters and instantiate the model
output_dim = 1000
d_model = 512
nhead = 8
num_decoder_layers = 6
dim_feedforward = 2048
dropout = 0.1

model = Seq2SeqModel(output_dim, d_model, nhead, num_decoder_layers, dim_feedforward, dropout)

# Example input data
memory = torch.randn(10, 32, d_model)  # (memory sequence length, batch size, d_model)
trg = torch.randint(0, output_dim, (20, 32))  # (target sequence length, batch size)

# Forward pass
output = model(memory, trg)
print(output.shape)  

RuntimeError: The shape of the 3D attn_mask is torch.Size([32, 20, 20]), but should be (256, 20, 20).

In [ ]:
transformer_decoder

In [84]:
out

tensor([[[ 0.1118, -1.1710, -0.8959,  ...,  2.2851, -1.6192, -0.4946],
         [ 0.8053, -0.6150, -1.0398,  ...,  2.0859, -1.3613, -0.5439],
         [ 0.7293, -0.3388, -1.3759,  ...,  1.4920, -2.3704, -0.8122],
         ...,
         [-0.0059, -0.7287, -0.5765,  ...,  1.6382, -1.9487, -0.1709],
         [ 0.4550, -0.9516, -1.5670,  ...,  1.5000, -1.2282, -0.3235],
         [ 0.8331, -0.7414, -1.4953,  ...,  1.4084, -1.8277, -0.1350]],

        [[ 0.0180, -1.0446, -1.0010,  ...,  1.6590, -1.4273, -0.2397],
         [ 0.7317, -0.9325, -1.5092,  ...,  1.6700, -1.7922, -1.0420],
         [-0.3930, -0.5197, -1.2167,  ...,  1.4408, -2.1300,  0.1562],
         ...,
         [-0.0971, -0.4941, -0.4797,  ...,  2.0280, -1.5911, -0.1578],
         [ 0.2414, -1.0646, -2.2118,  ...,  1.7787, -1.5203, -0.9634],
         [ 1.3117, -0.4621, -0.8127,  ...,  1.8086, -1.5333, -0.6950]],

        [[ 0.5540, -1.1116, -2.3528,  ...,  1.2588, -2.1234, -0.2447],
         [ 0.7436, -0.6004, -1.4084,  ...,  1

device(type='cuda')

In [ ]:
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    # Training data Iterator
    train_iter = Multi30k(split='train', language_pair=(SRC_LANGUAGE, TGT_LANGUAGE))
    # Create torchtext's Vocab object
    vocab_transform[ln] = build_vocab_from_iterator(yield_tokens(train_iter, ln),
                                                    min_freq=1,
                                                    specials=special_symbols,
                                                    special_first=True)

# Set ``UNK_IDX`` as the default index. This index is returned when the token is not found.
# If not set, it throws ``RuntimeError`` when the queried token is not found in the Vocabulary.
for ln in [SRC_LANGUAGE, TGT_LANGUAGE]:
    vocab_transform[ln].set_default_index(UNK_IDX)

In [4]:
# usando o tokenizer
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
from typing import List, Dict
from unidecode import unidecode
from nltk.tokenize.treebank import TreebankWordDetokenizer

class MyWordLevelTokenizer:
    def __init__(self):
        self.mode = 'train'
        self.special_tokens = {
            'PAD': 0,
            'UNK': 1,
            'SOS': 2,
            'EOF': 3
        }
        self.vocab = {**self.special_tokens}
        self.id_token = {v: k for k, v in self.vocab.items()}
        self.normalize: bool = True
    
    def encode(self, text: str) -> List[str]:
        if self.normalize:
            text = unidecode(text)

        tokens = word_tokenize(text, language='portuguese')

        if self.mode == 'train':
            for token in tokens:
                if token not in self.vocab:
                    id_ = max(self.vocab.values()) + 1
                    self.vocab[token] = id_
                    self.id_token[id_] = token
        return tokens

    def decoder(self, ids_list: List[int]):
        return TreebankWordDetokenizer().detokenize(ids_list)

    def tokens_to_ids(self, tokens_list: List[str]):
        return [self.vocab[token] for token in tokens_list]

    def ids_to_tokens(self, ids_list: List[int]):
        return [self.id_token[id_] for id_ in ids_list]

    def add_special_tokens(self, special_tokens: Dict[str, int]):
        self.special_tokens = {**special_tokens}
        return self.special_tokens

    def encoder_txt(self, text: str) -> List[int]:
        tokens_list = self.encode(text)
        # tokens_list = self.cut_seq(tokens_list)
        ids_list = self.tokens_to_ids(tokens_list)
        return ids_list

    def decoder_ids(self, ids_list: List[int]):
        tokens_list = self.ids_to_tokens(ids_list)
        print(tokens_list)
        return self.decoder(tokens_list)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\luiz_\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
d = MyWordLevelTokenizer()

In [6]:
# gerando o Dataset
import torch
import numpy as np
from torch.utils.data import Dataset, DataLoader

In [26]:
class GooglePlayAppsPt(Dataset):
    def __init__(self, data, tokenizer: MyWordLevelTokenizer, max_len: int = 124):
        self.data = data
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return self.data.num_rows

    def __getitem__(self, id_i):
        item = self.data[id_i]
        text = item['review']
        text_to_ids = self.tokenizer.encoder_txt(text)
        text_to_ids = self.cut_seq(text_to_ids)

        tokenized = torch.from_numpy(np.array(text_to_ids))
        tokenized = torch.tensor(tokenized, dtype=torch.long)
        
        decoder_input = tokenized[: self.max_len]
        desired_output = tokenized[1 : self.max_len + 1]
        return {'decoder_input': decoder_input, 'desired_output': desired_output}
        
    def cut_seq(self, ids_list: List[str]):
        len_token_list = len(ids_list)
        if len_token_list > self.max_len + 1:
            return ids_list[: self.max_len + 1]
        elif len_token_list < self.max_len + 1:
            return ids_list + [0] * (self.max_len + 1 - len_token_list)
        return ids_list

In [8]:
dataset

DatasetDict({
    train: Dataset({
        features: ['review', 'sentiment'],
        num_rows: 20000
    })
})

In [9]:
tokenizer=MyWordLevelTokenizer()

In [10]:
dataset_train = GooglePlayAppsPt(dataset['train'], tokenizer=tokenizer)

In [11]:
dataset_train[0]

C:\Users\luiz_\AppData\Local\Temp\ipykernel_20576\2861760060.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized = torch.tensor(tokenized, dtype=torch.long)


{'decoder_input': tensor([ 4,  5,  6,  7,  8,  9, 10,  4, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
         21, 22, 23, 24, 16, 25, 26,  4, 27, 28, 29, 30, 16, 31, 32, 33,  6, 34,
         28, 15, 16, 35, 36, 37, 38, 39, 40, 41, 31, 42, 43, 44, 28, 15, 16, 45,
         46, 47, 31, 48, 49, 16, 50, 45,  4,  5, 20, 51, 52, 53,  6, 54, 55,  4,
         56, 57,  5, 20, 58, 59, 60, 47, 40, 61, 62, 28, 15, 16, 63, 64, 26, 65,
          6, 63, 66, 26, 67, 68, 69, 70, 71, 72, 28, 73, 16, 31, 74, 75, 20, 76,
         38, 10, 77, 78, 29, 79, 10, 80, 81, 82, 44, 83, 84, 85, 86, 16]),
 'desired_output': tensor([ 5,  6,  7,  8,  9, 10,  4, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21,
         22, 23, 24, 16, 25, 26,  4, 27, 28, 29, 30, 16, 31, 32, 33,  6, 34, 28,
         15, 16, 35, 36, 37, 38, 39, 40, 41, 31, 42, 43, 44, 28, 15, 16, 45, 46,
         47, 31, 48, 49, 16, 50, 45,  4,  5, 20, 51, 52, 53,  6, 54, 55,  4, 56,
         57,  5, 20, 58, 59, 60, 47, 40, 61, 62, 28, 15, 16, 63, 64, 26, 65,  6,

In [12]:
# Dataloader
BATCH_SIZE = 16
dataloader = DataLoader(dataset_train, batch_size=BATCH_SIZE, shuffle=True)

In [13]:
# Model
import torch.nn as nn
import math

def generate_square_subsequent_mask(sz):
    """
    Generate a square mask for the sequence. The masked positions are filled with float('-inf').
    Unmasked positions are filled with float(0.0).
    """
    mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [14]:
class PositionalEncoding(nn.Module):
    def __init__(self, max_len, d_model, dropout=0.1):
        """
        :param max_len: Input length sequence.
        :param d_model: Embedding dimension.
        :param dropout: Dropout value (default=0.1)
        """
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=dropout)
        pe = torch.zeros(max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        pe = pe.unsqueeze(0)
        self.register_buffer('pe', pe)
    def forward(self, x):
        """
        Inputs of forward function
        :param x: the sequence fed to the positional encoder model (required).
        Shape:
            x: [sequence length, batch size, embed dim]
            output: [sequence length, batch size, embed dim]
        """
        x = x + self.pe[:, :x.size(1)]
        return self.dropout(x)

In [15]:
SEQUENCE_LENGTH = 124

class GooglePlayModel(nn.Module):
    def __init__(self, vocab_size, embed_dim, num_layers, num_heads):
        super(GooglePlayModel, self).__init__()
        self.pos_encoder = PositionalEncoding(max_len=SEQUENCE_LENGTH, d_model=embed_dim)
        self.emb = nn.Embedding(vocab_size, embed_dim)
        self.decoder_layer = nn.TransformerDecoderLayer(
            d_model=embed_dim, 
            nhead=num_heads, 
            batch_first=True
        )
        self.decoder = nn.TransformerDecoder(
            decoder_layer=self.decoder_layer,
            num_layers=num_layers,
        )
        self.linear = nn.Linear(embed_dim, vocab_size)
        self.dropout = nn.Dropout(0.2)
        
    # Positional encoding is required. Else the model does not learn.
    def forward(self, x):
        emb = self.emb(x)
        
        # Generate input sequence mask with shape (SEQUENCE_LENGTH, SEQUENCE_LENGTH)
        input_mask = generate_square_subsequent_mask(x.size(1)).to(x.device)
        
        x = self.pos_encoder(emb)
        x = self.decoder(x, memory=x, tgt_mask=input_mask, memory_mask=input_mask)
        x = self.dropout(x)
        out = self.linear(x)
        return out

In [16]:
for text in dataset['train']:
    tokenizer.encoder_txt(text['review'])

In [28]:
epochs = 100
learning_rate = 1e-5
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GooglePlayModel(
    vocab_size=len(tokenizer.vocab), 
    embed_dim=256,
    num_layers=2, 
    num_heads=4,
).to(device)

In [32]:
# Carregue o estado salvo, ignorando os parâmetros com tamanho diferente
checkpoint = torch.load('model/googleplay_model_9.pth', map_location=device)

# Crie uma nova state_dict ignorando os parâmetros com tamanho diferente
model_state_dict = model.state_dict()
for key in checkpoint.keys():
    if key in model_state_dict and checkpoint[key].size() == model_state_dict[key].size():
        model_state_dict[key] = checkpoint[key]
    else:
        print(f"Ignorando o parâmetro {key} devido ao tamanho incompatível.")

model.load_state_dict(model_state_dict)

Ignorando o parâmetro emb.weight devido ao tamanho incompatível.
Ignorando o parâmetro linear.weight devido ao tamanho incompatível.
Ignorando o parâmetro linear.bias devido ao tamanho incompatível.


<All keys matched successfully>

In [31]:
# Carregar o modelo completo salvo
model.load_state_dict(torch.load('model/googleplay_model_8.pth'))

# Colocar o modelo em modo de avaliação
# model.eval()

RuntimeError: Error(s) in loading state_dict for GooglePlayModel:
	size mismatch for emb.weight: copying a param with shape torch.Size([34853, 256]) from checkpoint, the shape in current model is torch.Size([34854, 256]).
	size mismatch for linear.weight: copying a param with shape torch.Size([34853, 256]) from checkpoint, the shape in current model is torch.Size([34854, 256]).
	size mismatch for linear.bias: copying a param with shape torch.Size([34853]) from checkpoint, the shape in current model is torch.Size([34854]).

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
print(model)
# Total parameters and trainable parameters.
total_params = sum(p.numel() for p in model.parameters())
print(f"{total_params:,} total parameters.")
total_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"{total_trainable_params:,} training parameters.\n")

GooglePlayModel(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (emb): Embedding(34853, 256)
  (decoder_layer): TransformerDecoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
    )
    (multihead_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=256, out_features=256, bias=True)
    )
    (linear1): Linear(in_features=256, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=256, bias=True)
    (norm1): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (norm3): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
    (dropout3): Dropout(p=0.1, inplace=False)
  )
  (decoder): 

In [17]:
for batch in dataloader:
    input_seq = batch['decoder_input'].to(device)
    target_seq = batch['desired_output'].to(device)
    break

C:\Users\luiz_\AppData\Local\Temp\ipykernel_14504\2861760060.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized = torch.tensor(tokenized, dtype=torch.long)


In [18]:
# Defina esta variável para capturar melhor os erros
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'

# Habilite a detecção de anomalias no grafo computacional
torch.autograd.set_detect_anomaly(True)

In [19]:
torch.cuda.empty_cache()

In [32]:
from pathlib import Path

In [35]:
Path('model').exists()

True

In [44]:
def callbacks():
    memory = {'loss': 0.0}

    def memory_function(model, loss, epoch, path: Path = 'model'):
        nonlocal memory
        if loss < memory['loss']:
            torch.save(model.state_dict(), path / Path(f'googleplay_model_{epoch}.pth'))
        memory.update({'loss': loss})
    return memory_function

In [45]:
callback = callbacks()

In [46]:
# Training
from tqdm import tqdm
def train(model, epochs, dataloader, criterion):
    model.train()
    epoch_loss = 0
    for epoch in range(epochs):
        torch.cuda.empty_cache()
        running_loss = 0
        batch_iterator = tqdm(dataloader, desc=f"Processing Epoch {epoch:02d}")
        for batch in batch_iterator:
            input_seq = batch['decoder_input'].to(device)
            target_seq = batch['desired_output'].to(device)
            outputs = model(input_seq)
            target_seq = target_seq.contiguous().view(-1)
            outputs = outputs.view(-1, len(tokenizer.vocab))
            
            loss = criterion(outputs, target_seq.view(-1))
            callback(model, loss.item(), epoch)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.detach().cpu().numpy()
            batch_iterator.set_postfix({"loss": f"{loss.item():6.3f}", "running loss": f"{running_loss:.3f}", "epoch loss": f"{epoch_loss:.3f}"})
        epoch_loss = running_loss / len(dataloader)
        print(f"Epoch {epoch} loss: {epoch_loss:.3f}")        

In [47]:
train(model, epochs, dataloader, criterion) 

Processing Epoch 00:   0%|                                                                    | 0/1250 [00:00<?, ?it/s]C:\Users\luiz_\AppData\Local\Temp\ipykernel_14504\2861760060.py:17: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tokenized = torch.tensor(tokenized, dtype=torch.long)
Processing Epoch 00: 100%|████| 1250/1250 [09:09<00:00,  2.27it/s, loss=3.979, running loss=5766.251, epoch loss=0.000]


Epoch 0 loss: 4.613


Processing Epoch 01: 100%|████| 1250/1250 [08:22<00:00,  2.49it/s, loss=3.679, running loss=4520.548, epoch loss=4.613]


Epoch 1 loss: 3.616


Processing Epoch 02: 100%|████| 1250/1250 [08:27<00:00,  2.46it/s, loss=3.558, running loss=4320.413, epoch loss=3.616]


...

Epoch 7 loss: 3.157


Processing Epoch 08: 100%|████| 1250/1250 [08:26<00:00,  2.47it/s, loss=2.898, running loss=3900.989, epoch loss=3.157]


Epoch 8 loss: 3.121


Processing Epoch 09:   2%|▏       | 20/1250 [00:08<08:29,  2.42it/s, loss=3.184, running loss=61.621, epoch loss=3.121]


KeyboardInterrupt: 

In [52]:
def cut_seq(ids_list: List[str]):
    len_token_list = len(ids_list)
    if len_token_list > 124:
        return ids_list[: 124]
    elif len_token_list < 124:
        return ids_list + [0] * (124 - len_token_list)
    return ids_list
    
def return_int_vector(text):
    tokens = tokenizer.encoder_txt(text)
    tokens = cut_seq(tokens)
    return torch.tensor(tokens, dtype=torch.int).unsqueeze(0)

def sample_next(predictions):
    """
    Greedy sampling.
    """
    # Greedy approach.
    probabilities = F.softmax(predictions[:, -1, :], dim=-1).cpu()
    next_token = torch.argmax(probabilities)
    return int(next_token.cpu())

def text_generator(sentence, generate_length):
    model.eval()
    sample = sentence
    for i in range(generate_length):
        int_vector = return_int_vector(sample)
        if len(int_vector) >= SEQUENCE_LENGTH - 1:
            break
        input_tensor = int_vector.to(device)
        with torch.no_grad():
            predictions = model(input_tensor)
        next_token = sample_next(predictions)
        # sample += ' ' + int_to_word[next_token]  # decode o token
        sample += ' ' + tokenizer.decoder_ids([next_token])
        print(sample)
    print('\n')

In [55]:
int_vector = return_int_vector('Eu gostaria')

In [56]:
int_vector

tensor([[34853,   485,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0]], dtype=torch.i

In [36]:
# Se disponível, obtenha informações sobre a GPU
if torch.cuda.is_available():
    print("Nome da GPU:", torch.cuda.get_device_name(0))
    print("Memória disponível:", torch.cuda.memory_allocated(0))

Nome da GPU: NVIDIA GeForce GTX 1650
Memória disponível: 543547392


In [57]:
input_tensor = int_vector.to(device)

In [58]:
model(input_tensor)

tensor([[[ 0.1724, -0.1588, -0.7231,  ..., -0.1175,  0.7343, -0.2147],
         [ 0.8182, -0.2380, -0.1657,  ..., -0.5185,  1.4670, -0.4672],
         [-0.0557, -0.2524, -0.4005,  ..., -0.6704,  1.3754, -0.1633],
         ...,
         [-0.0437, -0.1592, -0.2196,  ..., -0.7589,  0.3030, -0.2768],
         [-0.0635, -0.1781, -0.1911,  ..., -0.7686,  0.2920, -0.3342],
         [-0.0485, -0.2036, -0.1916,  ..., -0.7854,  0.3391, -0.3913]]],
       device='cuda:0', grad_fn=<ViewBackward0>)

In [54]:
import torch.nn.functional as F
sentences = [
    "Eu gostaria"
]
generate_length = 50
for sentence in sentences:
    print(f"PROMPT: {sentence}")
    text_generator(sentence, generate_length)

PROMPT: Eu gostaria
['deveras']
Eu gostaria deveras
['deveras']
Eu gostaria deveras deveras
['deveras']
Eu gostaria deveras deveras deveras
['deveras']
Eu gostaria deveras deveras deveras deveras
['deveras']
Eu gostaria deveras deveras deveras deveras deveras
['deveras']
Eu gostaria deveras deveras deveras deveras deveras deveras
['deveras']
Eu gostaria deveras deveras deveras deveras deveras deveras deveras
['deveras']
Eu gostaria deveras deveras deveras deveras deveras deveras deveras deveras
['deveras']
Eu gostaria deveras deveras deveras deveras deveras deveras deveras deveras deveras
['deveras']
Eu gostaria deveras deveras deveras deveras deveras deveras deveras deveras deveras deveras
['deveras']
Eu gostaria deveras deveras deveras deveras deveras deveras deveras deveras deveras deveras deveras
['deveras']
Eu gostaria deveras deveras deveras deveras deveras deveras deveras deveras deveras deveras deveras deveras
['deveras']
Eu gostaria deveras deveras deveras deveras deveras deve

In [52]:
return_int_vector("ola você")

tensor([[1033,  221]])

In [24]:
model

OrderedDict([('pos_encoder.pe',
              tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
                         0.0000e+00,  1.0000e+00],
                       [ 8.4147e-01,  5.4030e-01,  8.0196e-01,  ...,  1.0000e+00,
                         1.0746e-04,  1.0000e+00],
                       [ 9.0930e-01, -4.1615e-01,  9.5814e-01,  ...,  1.0000e+00,
                         2.1492e-04,  1.0000e+00],
                       ...,
                       [ 9.9882e-01, -4.8664e-02, -4.7778e-01,  ...,  9.9990e-01,
                         1.3002e-02,  9.9992e-01],
                       [ 4.9871e-01, -8.6677e-01,  4.1910e-01,  ...,  9.9990e-01,
                         1.3110e-02,  9.9991e-01],
                       [-4.5990e-01, -8.8797e-01,  9.7849e-01,  ...,  9.9990e-01,
                         1.3217e-02,  9.9991e-01]]], device='cuda:0')),
             ('emb.weight',
              tensor([[-0.8738, -0.1837,  1.4700,  ...,  0.3221, -0.1774,  0.6775],
         

In [ ]:
batch['decoder_input'].to(device)
batch['desired_output'].to(device)